In [6]:
import pandas as pd
import numpy as np
from minisom import MiniSom 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [16]:
df = pd.read_csv(r"C:\Users\julie\OneDrive\Skrivebord\Computational Data Analysis, 02582\Assignment 2\data\HR_data.csv", index_col=0)
df
# df = df.dropna(axis=1, how="any")
# df = df.sample(frac=1, random_state=42).reset_index(drop=True) # Random shuffle of rows

# target_cols = ["Cohort", "Phase", "Round"]
# X_cols = list(set(df.columns) - set(target_cols))

# X = df[X_cols]
# y = df[target_cols]

# test_size = 0.0
# test_split = len(X) - int(len(X) * test_size)

# X_train, y_train = np.array(X[:test_split]), np.array(y[:test_split])

,HR_TD_Mean,HR_TD_Median,HR_TD_std,HR_TD_Min,HR_TD_Max,HR_TD_AUC,HR_TD_Kurtosis,HR_TD_Skew,HR_TD_Slope_min,HR_TD_Slope_max,...,upset,hostile,alert,ashamed,inspired,nervous,attentive,afraid,active,determined
0,78.663878,76.700,7.480043,67.25,92.48,23048.450,-1.091448,0.369955,-0.730000,0.9775,...,1.0,1.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0
1,76.540732,76.610,2.584756,69.82,82.33,23959.920,-0.245338,0.338732,-0.360000,0.1900,...,2.0,1.0,3.0,2.0,2.0,2.0,3.0,1.0,3.0,3.0
2,78.173563,77.920,2.681255,72.22,82.80,20324.605,-0.615922,-0.233047,-0.630000,0.3575,...,1.0,1.0,2.0,1.0,3.0,2.0,3.0,2.0,3.0,3.0
3,83.073688,83.880,7.363598,69.42,96.12,24924.300,-0.866610,-0.046021,-0.465000,0.6500,...,1.0,1.0,2.0,1.0,3.0,2.0,3.0,2.0,3.0,3.0
4,72.281250,72.910,3.193762,64.95,79.98,23052.100,0.200401,-0.560948,-0.372500,0.3375,...,3.0,1.0,3.0,2.0,3.0,3.0,4.0,2.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,82.019065,78.830,8.794206,69.73,97.18,26244.705,-1.386324,0.251382,-0.517500,0.3525,...,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0
308,103.274248,104.625,14.208117,68.13,125.52,31523.695,-0.338091,-0.521254,-0.962500,1.0875,...,2.0,1.0,3.0,1.0,1.0,1.0,4.0,1.0,3.0,4.0
309,99.065000,97.490,10.079197,86.55,119.75,28235.230,-0.740136,0.629730,-0.657500,0.9175,...,1.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,3.0,4.0
310,91.793354,97.180,19.234169,60.90,120.67,28909.835,-1.229115,-0.181431,-0.717500,0.9800,...,2.0,1.0,3.0,1.0,3.0,1.0,4.0,1.0,4.0,4.0
